In [ ]:
import functools
import itertools
import math
import re
import statistics
from functools import reduce
from pprint import pprint
from typing import (
    List,
    Optional,
    Generator,
    NamedTuple,
    Tuple,
    Set,
    Any,
    Dict,
    Union,
    Callable,
    Iterable,
    Sequence,
)

import rich.panel
import sklearn.base
from alive_progress import alive_bar
from numpy.distutils.misc_util import is_sequence
from tqdm import tqdm, trange
from log_symbols import LogSymbols

import overrides as overrides
import pandas as pds
import tabulate as tabulate
from collections import namedtuple
from sklearn import linear_model, metrics, preprocessing, model_selection, svm
from tabulate import tabulate
from pathlib import Path
import pickle
from halo import Halo
import random
import argparse
import sys
from sklearn.experimental import enable_halving_search_cv
from sklearn import neural_network, tree
from sklearn.model_selection import (
    HalvingGridSearchCV,
    RandomizedSearchCV,
    HalvingRandomSearchCV,
)
import scipy.stats as scistats

import rich.progress
import commons
from data_types import PickleOut
from rich.traceback import install as niceTracebacks
from rich.table import Table as rTable

from db_actions import db_actions
import numpy as np

from linRegPred import getSplits, getTransformers
from scipy.stats import rankdata
from sklearn.metrics import r2_score
import textdistance as TD

In [ ]:
with db_actions.connect() as conn:
    dF = pds.read_sql('SELECT * FROM "runtimeScorePredBase1000"', conn)
# print(dF)
all_x_cols = [
    "build-linux-kernel1",
    "fio2",
    "fio3",
    "fio4",
    "fio5",
    "fio6",
    "fio7",
    "fio8",
    "fio9",
    "iperf10",
    "iperf11",
    "iperf12",
    "iperf13",
    "john-the-ripper14",
    "john-the-ripper15",
    "ramspeed16",
    "ramspeed17",
    "ramspeed18",
    "ramspeed19",
    "ramspeed20",
    "ramspeed21",
    "ramspeed22",
    "ramspeed23",
    "ramspeed24",
    "ramspeed25",
    "stream26",
    "stream27",
    "stream28",
    "stream29",
    "pCpu",
    "cpus",
    "rss",
    "vmem",
    "rchar",
    "wchar",
    "syscr",
    "syscw",
]
filtered_x_cols = ['john-the-ripper15', 'ramspeed17', 'ramspeed18', 'ramspeed19', 'ramspeed20', 'ramspeed21',
                   'ramspeed23', 'ramspeed24', 'ramspeed25', 'pCpu', 'cpus', 'vmem', 'rchar', 'wchar', 'syscr',
                   'syscw']
x_cols = all_x_cols
y_cols = "realtime"

X: pds.DataFrame = dF[x_cols]
y: pds.DataFrame = dF[y_cols]

In [ ]:
from my_yaml import yaml_load

with open("nodeConfigIdLookup.yaml", "r") as f:
    t = yaml_load(f)
    nodeIDLUT = {v: k for k, v in t.items()}
dF["nodeName"] = dF["nodeConfig"].transform(lambda x: nodeIDLUT[x])

In [ ]:
awsGroupLUT = {
    "general": ["M7g", "Mac", "M6g", "M6i", "M6in", "M6a", "M5", "M5n", "M5zn", "M5a", "M4", "A1", "T4g", "T3", "T3a",
                "T2"],
    "compute": ["C7g", "C7gn", "C6i", "C6in", "C6a", "C6g", "C6gn", "C5", "C5n", "C5a", "C4"],
    "memory": ["R7g", "R7iz", "R6g", "R6i", "R6in", "R6a", "R5", "R5n", "R5b", "R5a", "R4", "X2gd", "X2idn", "X2iedn",
               "X2iezn", "X1", "X1e", "z1d"],
    "accelerated": ["P4", "P3", "P2", "DL1", "Trn1", "Inf2", "Inf1", "G5", "G5g", "G4dn", "G4ad", "G3", "F1", "VT1"],
    "storage": ["Im4gn", "Is4gen", "I4i", "I3", "I3en", "D2", "D3", "D3en", "H1"],
    "HPC": ["Hpc6id", "Hpc6a"]
}

In [ ]:
awsInstanceGroupLUT = {i.lower(): k for k, v in awsGroupLUT.items() for i in v}

In [ ]:
dF["nodeGroup"] = dF["nodeName"].transform(lambda x: awsInstanceGroupLUT[x.split(".")[0]])

In [ ]:
for x in dF.groupby(by=["nodeGroup", "wfName", "taskName"]):
    c, d = x
    print(c, d.realtime.mean(), 100 * d.realtime.std() / d.realtime.mean(),
          100 * (d.realtime.max() - d.realtime.min()) / d.realtime.mean())

In [ ]:
import seaborn as sns

#sns.boxplot(data=dF,y="realtime",x="taskName",hue="nodeGroup")
for wf in dF.groupby(by="wfName"):
    wfname, data = wf
    print(wfname)
    grid = sns.FacetGrid(data, row="wfName", col="nodeGroup", sharex=False, sharey=True, margin_titles=True, height=5)
    grid.map_dataframe(sns.boxplot, y="realtime", x="taskName")

# Find prototypes for each group

#### failed attempt (i think?)

In [ ]:
#means = dF.groupby(by=["nodeGroup", "wfName", "taskName"]).realtime.mean().reset_index()
accu = dict()
for g in dF.groupby(by=["nodeGroup", "wfName", "taskName"]):
    l, d = g
    div = (d.realtime - d.realtime.mean()) ** 2 * d.realtime.mean()
    for i, nN in enumerate(d.nodeName.values):
        if nN not in accu.keys():
            accu[nN] = 0
        accu[nN] += div.values[i]



In [ ]:
print(accu)

#### good attempt

In [ ]:
with db_actions.connect() as conn:
    scores_df = pds.read_sql('SELECT * FROM "nodeBenchmarkTransposedScores"', conn)


In [ ]:
scores_df["nodeName"] = scores_df["nodeConfig"].transform(lambda x: nodeIDLUT[x])
scores_df["nodeGroup"] = scores_df["nodeName"].transform(lambda x: awsInstanceGroupLUT[x.split(".")[0]])

In [ ]:
distances = dict()
benchmarkCols = [
    "build-linux-kernel1",
    "fio2",
    "fio3",
    "fio4",
    "fio5",
    "fio6",
    "fio7",
    "fio8",
    "fio9",
    "iperf10",
    "iperf11",
    "iperf12",
    "iperf13",
    "john-the-ripper14",
    "john-the-ripper15",
    "ramspeed16",
    "ramspeed17",
    "ramspeed18",
    "ramspeed19",
    "ramspeed20",
    "ramspeed21",
    "ramspeed22",
    "ramspeed23",
    "ramspeed24",
    "ramspeed25",
    "stream26",
    "stream27",
    "stream28",
    "stream29"]
for c in itertools.product(scores_df.query("nodeGroup=='general'").nodeName.values,
                           scores_df.query("nodeGroup=='compute'").nodeName.values,
                           scores_df.query("nodeGroup=='memory'").nodeName.values,
                           scores_df.query("nodeGroup=='storage'").nodeName.values):
    n1, n2, n3, n4 = c
    ns1 = scores_df.query("nodeName == @n1")[benchmarkCols].values[0]
    ns2 = scores_df.query("nodeName == @n2")[benchmarkCols].values[0]
    ns3 = scores_df.query("nodeName == @n3")[benchmarkCols].values[0]
    ns4 = scores_df.query("nodeName == @n4")[benchmarkCols].values[0]
    distances[c] = np.linalg.norm(ns1 - ns2) + np.linalg.norm(ns1 - ns3) + np.linalg.norm(ns1 - ns4) + np.linalg.norm(
        ns2 - ns3) + np.linalg.norm(ns2 - ns4) + np.linalg.norm(ns3 - ns4)

In [ ]:
cs, ds = distances.keys(), distances.values()
#print("\n".join(map(lambda x: repr(x), sorted(zip(cs, ds), key=lambda x: x[1])[:10])))
groupPrototypes = sorted(zip(cs, ds), key=lambda x: x[1])[0][0]
print(groupPrototypes)

# Produce Group Rankings

In [117]:
groupPerformances = dict()
for g in dF.groupby(by=["nodeGroup", "wfName", "taskName"]):
    l, d = g
    ng, wfn, tn = l
    if (wfn, tn) not in groupPerformances.keys():
        groupPerformances[(wfn, tn)] = dict()
    groupPerformances[(wfn, tn)][ng] = d.realtime.mean()
    #groupPerformances[l] = d.realtime.mean()

In [119]:
groupOrder = ["general", "compute", "memory", "storage"]

In [120]:
groupRankings = pds.DataFrame(columns=["wfName", "taskName", *groupOrder])

In [124]:
for k, v in groupPerformances.items():
    wfn, tn = k
    performances = [v[i] for i in groupOrder]
    ranks = rankdata(performances, method="min")
    groupRankings = groupRankings.append({"wfName": wfn, "taskName": tn, **{o: r for o, r in zip(groupOrder, ranks)}},
                                         ignore_index=True)

In [126]:
print(groupRankings)

                   wfName                                           taskName  \
0    nfcore/chipseq:1.2.2                                          BWA_INDEX   
1    nfcore/chipseq:1.2.2                                            BWA_MEM   
2    nfcore/chipseq:1.2.2                                    CONSENSUS_PEAKS   
3    nfcore/chipseq:1.2.2                           CONSENSUS_PEAKS_ANNOTATE   
4    nfcore/chipseq:1.2.2                             CONSENSUS_PEAKS_DESEQ2   
..                    ...                                                ...   
74  nfcore/viralrecon:2.1  NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:NEXTC...   
75  nfcore/viralrecon:2.1  NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:PANGOLIN   
76  nfcore/viralrecon:2.1  NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:PLOT_...   
77  nfcore/viralrecon:2.1     NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:QUAST   
78  nfcore/viralrecon:2.1  NFCORE_VIRALRECON:ILLUMINA:VARIANTS_IVAR:SNPEF...   

   general compute memory storage  
0  

#### Produce inter-group node rankings

In [131]:
nodeOrder = list(pds.unique(dF.nodeName))

In [140]:
groupNodePerformances = dict()
for g in dF.groupby(by=["nodeGroup", "wfName", "taskName", "nodeName"]):
    l, d = g
    ng, wfn, tn, nn = l
    l1 = (ng, wfn, tn)
    if l1 not in groupNodePerformances.keys():
        groupNodePerformances[l1] = dict()
    groupNodePerformances[l1][nn] = d.realtime.mean()
    #groupNodePerformances[(ng, wfn, tn)] = d.sort_values(by="nodeName", key=lambda v: sorted(list(v), key=lambda x: nodeOrder.index(x))).realtime.values

    #groupNodePerformances[l] = d.realtime.mean()

In [141]:
print(groupNodePerformances)

{('compute', 'nfcore/chipseq:1.2.2', 'BWA_INDEX'): {'c5.2xlarge': 6490.4, 'c5.xlarge': 9775.4, 'c5a.2xlarge': 6593.2, 'c5a.xlarge': 5716.4}, ('compute', 'nfcore/chipseq:1.2.2', 'BWA_MEM'): {'c5.2xlarge': 3933.0333333333333, 'c5.xlarge': 4933.633333333333, 'c5a.2xlarge': 4114.666666666667, 'c5a.xlarge': 3796.0666666666666}, ('compute', 'nfcore/chipseq:1.2.2', 'CONSENSUS_PEAKS'): {'c5.2xlarge': 1504.8, 'c5.xlarge': 1594.2, 'c5a.2xlarge': 1446.8, 'c5a.xlarge': 1399.0}, ('compute', 'nfcore/chipseq:1.2.2', 'CONSENSUS_PEAKS_ANNOTATE'): {'c5.2xlarge': 3566.2, 'c5.xlarge': 3873.4, 'c5a.2xlarge': 3458.6, 'c5a.xlarge': 3417.4}, ('compute', 'nfcore/chipseq:1.2.2', 'CONSENSUS_PEAKS_DESEQ2'): {'c5.2xlarge': 11615.8, 'c5.xlarge': 12750.8, 'c5a.2xlarge': 11267.0, 'c5a.xlarge': 11170.6}, ('compute', 'nfcore/chipseq:1.2.2', 'FASTQC'): {'c5.2xlarge': 6440.1, 'c5.xlarge': 7184.7, 'c5a.2xlarge': 6673.5, 'c5a.xlarge': 6797.666666666667}, ('compute', 'nfcore/chipseq:1.2.2', 'MACS2'): {'c5.2xlarge': 2009.1, 

In [142]:
groupNodeRankings = pds.DataFrame(
    data={"wfName": dF.wfName, "taskName": dF.taskName, **{nn: [0]*len(dF) for nn in nodeOrder}})  #columns=["wfName", "taskName", *nodeOrder])


In [144]:
print(groupNodeRankings)

                      wfName  \
0       nfcore/chipseq:1.2.2   
1       nfcore/chipseq:1.2.2   
2       nfcore/chipseq:1.2.2   
3       nfcore/chipseq:1.2.2   
4       nfcore/chipseq:1.2.2   
...                      ...   
33340  nfcore/viralrecon:2.1   
33341  nfcore/viralrecon:2.1   
33342  nfcore/viralrecon:2.1   
33343  nfcore/viralrecon:2.1   
33344  nfcore/viralrecon:2.1   

                                                taskName  m5zn.2xlarge  \
0                                  get_software_versions             0   
1                                  get_software_versions             0   
2                                              BWA_INDEX             0   
3                                                 FASTQC             0   
4                                              BWA_INDEX             0   
...                                                  ...           ...   
33340  NFCORE_VIRALRECON:ILLUMINA:VARIANTS_BCFTOOLS:P...             0   
33341  NFCORE_VIRALRECO

In [ ]:
for k, v in groupNodePerformances.items():
    ng, wfn, tn = k
    vk = v.keys()
    performances = v.values()
    ranks = rankdata(performances, method="min")
    groupNodeRankings.loc


In [143]:
print(groupNodeRankings)

                      wfName  \
0       nfcore/chipseq:1.2.2   
1       nfcore/chipseq:1.2.2   
2       nfcore/chipseq:1.2.2   
3       nfcore/chipseq:1.2.2   
4       nfcore/chipseq:1.2.2   
...                      ...   
33340  nfcore/viralrecon:2.1   
33341  nfcore/viralrecon:2.1   
33342  nfcore/viralrecon:2.1   
33343  nfcore/viralrecon:2.1   
33344  nfcore/viralrecon:2.1   

                                                taskName  m5zn.2xlarge  \
0                                  get_software_versions             0   
1                                  get_software_versions             0   
2                                              BWA_INDEX             0   
3                                                 FASTQC             0   
4                                              BWA_INDEX             0   
...                                                  ...           ...   
33340  NFCORE_VIRALRECON:ILLUMINA:VARIANTS_BCFTOOLS:P...             0   
33341  NFCORE_VIRALRECO